# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [ ]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

hotel_function = {
    "name": "get_hotel_details_based_on_city",
    "description": "Get the details of a hotel from the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": hotel_function},
]

In [ ]:
tools

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    elif tool_call.function.name == "get_hotel_details_based_on_city":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        hotel_details = get_hotel_details_based_on_city(city)
        response = {
            "role": "tool",
            "content": hotel_details,
            "tool_call_id": tool_call.id
        }
    else:
        response = {
            "role": "tool",
            "content": "Unknown tool",
            "tool_call_id": tool_call.id
        }
    return response

In [ ]:
gr.ChatInterface(fn=chat).launch()

## Let's make a couple of improvements

Handling multiple tool calls in 1 response

Handling multiple tool calls 1 after another

In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
def handle_tool_calls(message):
    print(message)
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get("destination_city")
            price_details = get_ticket_price(city)
            responses.append(
                {
                    "role": "tool",
                    "content": price_details,
                    "tool_call_id": tool_call.id,
                }
            )
        elif tool_call.function.name == "get_hotel_details_based_on_city":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get("destination_city")
            hotel_details = get_hotel_details_based_on_city(city)
            responses.append(
                {
                    "role": "tool",
                    "content": hotel_details,
                    "tool_call_id": tool_call.id,
                }
            )
    return responses

In [ ]:
gr.ChatInterface(fn=chat).launch()

In [ ]:
def _print_full_response(response):
    """Print full API response in readable format (no truncation)."""
    data = response.model_dump()
    print("=== Full response ===")
    print(json.dumps(data, indent=2, default=str, ensure_ascii=False))
    msg = response.choices[0].message if response.choices else None
    if msg and getattr(msg, "tool_calls", None):
        print("\n=== All tool_calls (full) ===")
        for i, tc in enumerate(msg.tool_calls):
            print(f"\n--- Tool call {i + 1} ---")
            print(json.dumps(tc.model_dump(), indent=2, default=str, ensure_ascii=False))

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    _print_full_response(response)
    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [ ]:
import sqlite3


In [ ]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    cursor.execute('CREATE TABLE IF NOT EXISTS hotel (city TEXT PRIMARY KEY, date DATE, price REAL,type TEXT,status TEXT)')
    conn.commit()

In [ ]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [ ]:
def get_hotel_details_based_on_city(city):
    print(f"DATABASE TOOL CALLED: Getting hotel_details for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT city, date, price, type, status FROM hotel WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()

    if not result:
        return "No Hotel data available for this city"

    city_db, date, price, room_type, status = result
    return (
        f"Hotel in {city_db.title()} on {date}: "
        f"${price} ({room_type}, status: {status})"
    )

In [ ]:
get_hotel_details_based_on_city("london")

In [ ]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [ ]:
ticket_prices = {"london":799, "paris": 899, "tokyo": 1420, "sydney": 2999}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

In [ ]:
# Populate the `hotel` table created above
# Schema: hotel (city TEXT PRIMARY KEY, date DATE, price REAL, type TEXT, status TEXT)

hotel_data = [
    ("london", "2026-02-01", 180.0, "standard", "available"),
    ("paris", "2026-02-05", 210.0, "deluxe", "available"),
    ("tokyo", "2026-03-10", 250.0, "standard", "sold_out"),
    ("sydney", "2026-04-15", 300.0, "suite", "available"),
]

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.executemany(
        """
        INSERT INTO hotel (city, date, price, type, status)
        VALUES (?, ?, ?, ?, ?)
        ON CONFLICT(city) DO UPDATE SET
            date = excluded.date,
            price = excluded.price,
            type = excluded.type,
            status = excluded.status
        """,
        hotel_data,
    )
    conn.commit()

In [ ]:
get_hotel_details_based_on_city(city)

In [ ]:
gr.ChatInterface(fn=chat).launch()

## Exercise

Add a tool to set the price of a ticket!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Hopefully this hardly needs to be stated! You now have the ability to give actions to your LLMs. This Airline Assistant can now do more than answer questions - it could interact with booking APIs to make bookings!</span>
        </td>
    </tr>
</table>